<a href="https://colab.research.google.com/github/gosaitos/GEN-AI/blob/main/Text_to_video_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from diffusers import DiffusionPipeline
import torch
import numpy as np
import imageio
from PIL import Image

# Load pipeline
pipe = DiffusionPipeline.from_pretrained(
    "damo-vilab/text-to-video-ms-1.7b",
    torch_dtype=torch.float16,
    variant="fp16"
).to("cuda")

# Generate batched video frames
prompt = input('enter a text:')
video_batches = pipe(prompt, num_inference_steps=50).frames

# Frame upscaling target
target_resolution = (512, 512)

# Flatten & process frames
processed_frames = []

for i, batch in enumerate(video_batches):
    for j, frame in enumerate(batch):
        # Normalize and convert to uint8
        if frame.dtype != np.uint8:
            frame = (frame * 255).clip(0, 255).astype(np.uint8)

        # Ensure RGB format
        if frame.ndim == 2:
            frame = np.stack([frame] * 3, axis=-1)
        elif frame.ndim == 3 and frame.shape[2] == 1:
            frame = np.repeat(frame, 3, axis=2)
        elif frame.ndim == 3 and frame.shape[2] > 4:
            frame = frame[:, :, :3]

        # Convert to PIL for resizing
        image = Image.fromarray(frame)
        image = image.resize(target_resolution, Image.LANCZOS)
        processed_frames.append(np.array(image))

print(f"✅ Total processed frames: {len(processed_frames)}")

# Save high-quality MP4 using FFmpeg writer
output_path = "output.mp4"
writer = imageio.get_writer(
    output_path,
    fps=10,
    codec='libx264',          # High quality H.264 codec
    bitrate="10M",             # Increase bitrate for better quality
    quality=10                # Max quality
)

for frame in processed_frames:
    writer.append_data(frame)
writer.close()

print(f"video saved as {output_path}")

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

enter a text:JACKIE CHAN


  0%|          | 0/50 [00:00<?, ?it/s]

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2136, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1662, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^

✅ Total processed frames: 16
video saved as output.mp4
